## Preparation

In [ ]:
%%capture
!pip install pytorch-lightning
# !pip install torchmetrics
# !pip install tqdm

In [ ]:
# Setting up google drive 
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
%cd '/content/gdrive/MyDrive/Colab Notebooks/code'

Mounted at /content/gdrive
/content/gdrive/MyDrive/Colab Notebooks/code


In [ ]:
import pandas as pd
import numpy as np
import os.path
from IPython.utils import io

import torch
from torch import nn
from torch.nn.functional import interpolate
from torch.utils.data import Dataset, DataLoader
import torch.autograd as autograd
import torch.nn.functional as F
import torch.optim as optim

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix

import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger
import torchmetrics
from torchmetrics.functional import accuracy

import seaborn as sns
from pylab import rcParams
from tqdm.auto import tqdm

from project_Prepro import preprocess_XY, preprocess_Y, preprocess_Y2, Split_Interpolate_CNN, Split_Pad_CNN
from project_Data import PhyAAtDataModule
from project_CNN import CNNmodel, flatten_size

In [ ]:
!python --version

Python 3.7.13


In [ ]:
print('numpy version:',np.__version__)
print('pandas version:',pd.__version__)
print('lightning version:',pl.__version__)
print('torch version:',torch.__version__)

numpy version: 1.21.6
pandas version: 1.3.5
lightning version: 1.7.1
torch version: 1.12.1+cu113


## Within-Subject Experiment

In [ ]:
# task label - task mapping
task_label = {
    1:'Correctness_label',
    2:'SNRdB_label',
    3:'Semanticity_label'
    }

# dataframe to save all results
ls = ['S{}'.format(i) for i in range(1,26)]
col = ['Task{}'.format(i) for i in range(1,4)]
result = pd.DataFrame(index = ls, columns = col)

BATCH_SIZE = 20
N_EPOCHS = 100

In [ ]:
def init_weights(m):
    if isinstance(m, nn.Conv1d) or isinstance(m, nn.Linear):
        torch.nn.init.kaiming_uniform_(m.weight)
        torch.nn.init.constant_(m.bias,0)

# default initial weighting of Conv1d and linear is from U(-sqrt(k), sqrt(k)) 

In [ ]:
# loop over all subjects
# for all subjects change 2 to 26
for subject_No in range(1,2): 
    print("Subject No." + str(subject_No) + ":")
    path = 'X{}_trial.csv'.format(subject_No)
    Y = pd.read_csv('S{0}_Textscore.csv'.format(subject_No))

    # 1 Data Input and process
    if os.path.exists(path):
      X = pd.read_csv(path)
      Y = preprocess_Y(subject_No)
    else:
      X, Y = preprocess_XY(subject_No)
      X.to_csv('X{}_trial.csv'.format(subject_No),index = False)

    FEATURE_COLUMNS = X.columns[1:15].tolist()

    # loop over all tasks
    # for all tasks change 2 to 4
    for target_task in range(1,2): 
        print("Task:",target_task)
        target_label = task_label[target_task]

        # 2 Interpolate, Split into trials, and split into train, valid, and test sequences
        X_trial, max_trial_length = Split_Interpolate_CNN(X,Y,target_label)
        # X_trial, max_trial_length = Split_Pad_CNN(X,Y,target_label) # pad instead of interpolate as an alternative shoice
        
        train_sequences, test_sequences = train_test_split(X_trial, test_size = 0.25)
        valid_sequences, test_sequences = train_test_split(test_sequences, test_size = 0.4) # train:valid:test = 75:15:10

        # 3 Dataset DataModuel
        data_module = PhyAAtDataModule(train_sequences,valid_sequences, test_sequences, BATCH_SIZE)
        
        # 4 Net and Model

        # get tensor size after flatten to establish corresponding CNN structure to cope with varied length
        dummy = torch.rand(size=(1, 14, max_trial_length), dtype=torch.float32)
        net = nn.Sequential(
        nn.BatchNorm1d(14)
        ,nn.Conv1d(14, 84, kernel_size = 128*3, stride = 2, padding = 0)
        ,nn.AvgPool1d(kernel_size = 64, stride = 2)
        ,nn.Conv1d(84, 200, kernel_size = 32, stride = 4, padding = 0)
        ,nn.AvgPool1d(kernel_size = 4, stride = 8)
        ,nn.LeakyReLU()
        ,nn.Dropout()
        ,nn.Flatten()
        )
        flatten_size = net(dummy).shape[1]

        model = CNNmodel(
            num_features = len(FEATURE_COLUMNS),
            num_classes = len(Y[task_label[target_task]].unique()),
            flatten_size = flatten_size
        )
        model = model.apply(init_weights);
        
        # 5 Train
        earlystopping_callback = EarlyStopping(
            monitor = 'val_accuracy',
            mode = 'max',
            patience = 10
        )
        checkpoint_callback = ModelCheckpoint(
            # dirpath = 'lightning_logs/checkpoints',
            filename = '{epoch:02d}-{val_accuracy:.2f}-{val_loss:.2f}' ,
            save_top_k = 1,
            verbose = True,
            monitor = 'val_accuracy',
            mode = "max"
        )
        callback_list = [checkpoint_callback, earlystopping_callback]

        trainer = pl.Trainer(
        #     logger=logger,
            callbacks=callback_list,
            max_epochs=N_EPOCHS,
            # ,accelerator='cpu'
            accelerator='gpu',
            devices = 1
            # ,auto_lr_find=True
        )
        
        trainer.fit(model, data_module)

        # 6 test and result
        test_result = trainer.test(model, dataloaders = data_module.test_dataloader())
        result.at["S{}".format(subject_No), "Task{}".format(target_task)] = round(test_result[0]['test_accuracy'],2)

Subject No.1:
Task: 1


INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name      | Type             | Params
-----------------------------------------------
0 | model     | CNNnet           | 1.6 M 
1 | criterion | CrossEntropyLoss | 0     
-----------------------------------------------
1.6 M     Trainable params
0         Non-trainable params
1.6 M     Total params
6.285     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/trainer.py:1898: PossibleUserWarning: The number of training batches (6) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  category=PossibleUserWarning,


Training: 0it [00:00, ?it/s]

...

INFO:pytorch_lightning.utilities.rank_zero:Epoch 16, global step 102: 'val_accuracy' was not in top 1


Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:Epoch 17, global step 108: 'val_accuracy' was not in top 1
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      test_accuracy         0.6000000238418579
        test_loss            4.567758083343506
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


In [ ]:
result.at['Avg_Acc'] = round(result.mean(axis = 0, skipna = True),2)
result

,Task1,Task2,Task3
S1,0.27,0.53,0.47
S2,0.33,0.2,0.6
S3,0.6,0.6,0.53
S4,0.4,0.27,0.6
S5,0.53,0.53,0.53
S6,0.67,0.6,0.53
S7,0.47,0.47,0.6
S8,0.4,0.6,0.47
S9,0.6,0.47,0.6
S10,0.4,0.53,0.6


In [ ]:
# result.to_csv('result_cnn_interpolate.csv')
# result.to_csv('result_cnn_pad.csv')